In [1]:
'''
tcloud中用销售订单生成税局发票模板
'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd

In [2]:
def danjuChuli(df):
    df = df.dropna(how='all', axis=0)
    cols = [i for i in df.columns.to_list() if 'Unnamed' not in i]
    df = df[cols]
    # get maxrows
    for label, ser in df.items():
        for num, x in enumerate(ser):
            if isinstance(x, str):
                if '合计' in x:
                    index = num
                    print(index)
                    break
    df = df.iloc[:index]
    df = df.dropna(how='all', axis=1)
    return df
def chuliMingchen(d):
    d['存货名称'] = d['存货名称'].str.replace('运费',"")
    qian = d['存货名称'].str.split('型').str[0]
    hou = d['存货名称'].str.split('型').str[1]
    qian = pd.Series(qian).fillna('')
    hou = pd.Series(hou).fillna('')
    return qian,hou

In [3]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型
0,N0785（电商）a牛卡缝线练习本,1060202010000000000,16K60型,本,6000.0,0.8,4800,0.01,NaN,NaN
1,YS定制（混装）牛卡缝线簿,1060202010000000000,16K60型,本,12000.0,0.8,9600,0.01,NaN,NaN
2,TG78系列混装牛卡缝线本（方格双竖线英语）,1060202010000000000,16K60型,本,12000.0,0.8,9600,0.01,NaN,NaN
3,TG78系列混装牛卡缝线本（方格双竖线英语）,1060202010000000000,16K60型,本,6000.0,0.8,4800,0.01,NaN,NaN
4,运费,NaN,NaN,NaN,NaN,NaN,-540,NaN,NaN,NaN


In [5]:
fname_dingdan = r"F:\repos\fish\肥猫销售订单SO-2024-04-0082.xlsx"
df_dingdan = pd.read_excel(fname_dingdan,sheet_name = 0,header = 8)
df_dingdan

,Unnamed: 0,序号,货号,Unnamed: 3,Unnamed: 4,存货名称,Unnamed: 6,Unnamed: 7,件含量,数量（件）,数量（本）,Unnamed: 11,单价,扣率（%）,金额（元）
0,NaN,1,N0785（电商）a,NaN,NaN,16K60型牛卡缝线练习本,NaN,NaN,200.0,30.0,6000,NaN,0.8,1,4800.0
1,NaN,2,YS定制（混装）,NaN,NaN,16K60型牛卡缝线簿,NaN,NaN,200.0,60.0,12000,NaN,0.8,1,9600.0
2,NaN,3,TG78系列混装,NaN,NaN,16K60型牛卡缝线本（方格双竖线英语）,NaN,NaN,200.0,60.0,12000,NaN,0.8,1,9600.0
3,NaN,4,TG78系列混装,NaN,NaN,16K60型牛卡缝线本（方格双竖线英语）,NaN,NaN,200.0,30.0,6000,NaN,0.8,1,4800.0
4,NaN,5,运费,NaN,NaN,运费,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,-540.0
5,NaN,合计,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,36000.00,NaN,NaN,NaN,28260.0
6,NaN,制单人：,NaN,高婷,客户：,NaN,业务员：,NaN,NaN,NaN,物流：,NaN,NaN,超授信额度：,NaN


In [6]:
df_dingdan1 = danjuChuli(df_dingdan) 
qian,hou = chuliMingchen(df_dingdan1)
df_dingdan1

5


,序号,货号,存货名称,件含量,数量（件）,数量（本）,单价,扣率（%）,金额（元）
0,1,N0785（电商）a,16K60型牛卡缝线练习本,200.0,30.0,6000,0.8,1,4800.0
1,2,YS定制（混装）,16K60型牛卡缝线簿,200.0,60.0,12000,0.8,1,9600.0
2,3,TG78系列混装,16K60型牛卡缝线本（方格双竖线英语）,200.0,60.0,12000,0.8,1,9600.0
3,4,TG78系列混装,16K60型牛卡缝线本（方格双竖线英语）,200.0,30.0,6000,0.8,1,4800.0
4,5,运费,,NaN,NaN,0,NaN,NaN,-540.0


In [ ]:
df_fapiao.columns.to_list()

In [7]:
df_dingdan1['项目名称'] = df_dingdan1['货号'] + '-' + hou
df_dingdan1['项目名称'] = df_dingdan1['项目名称'].str.replace('运费-','运费')
df_dingdan1['商品和服务税收分类编码'] = '1060202010000000000'
df_dingdan1['规格型号'] = qian
df_dingdan1['单位'] = '本'
df_dingdan1['商品数量'] = df_dingdan1['数量（本）']
df_dingdan1['商品单价'] = df_dingdan1['单价']
df_dingdan1['金额'] = df_dingdan1['金额（元）']
df_dingdan1['税率'] = 0.01
df_dingdan1['折扣金额'] = ''
df_dingdan1['优惠政策类型'] = ''

df_dingdan2 = df_dingdan1[['项目名称',
 '商品和服务税收分类编码',
 '规格型号',
 '单位',
 '商品数量',
 '商品单价',
 '金额',
 '税率',
 '折扣金额',
 '优惠政策类型']]
df_dingdan2

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型
0,N0785（电商）a-牛卡缝线练习本,1060202010000000000,16K60,本,6000,0.8,4800.0,0.01,,
1,YS定制（混装）-牛卡缝线簿,1060202010000000000,16K60,本,12000,0.8,9600.0,0.01,,
2,TG78系列混装-牛卡缝线本（方格双竖线英语）,1060202010000000000,16K60,本,12000,0.8,9600.0,0.01,,
3,TG78系列混装-牛卡缝线本（方格双竖线英语）,1060202010000000000,16K60,本,6000,0.8,4800.0,0.01,,
4,运费,1060202010000000000,,本,0,NaN,-540.0,0.01,,


In [8]:
fname_result = r"F:\repos\fish\发票模板 - 副本.xlsx"

with pd.ExcelWriter(fname_result, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
  
    df_dingdan2.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)






In [ ]:
#提取发票台头等
fname_taitou = r"F:\repos\fish\肥猫销售订单SO-2024-04-0082.xlsx"
df_taitou = pd.read_excel(fname_dingdan,sheet_name = 0)
df_taitou1 = df_taitou.iloc[:,[1,3]]
df_taitou1.columns = ['kehu','names' ]
for i,j in zip(df_taitou1['kehu'],df_taitou1['names']):
    if i == '发货单号：':
        danhao = j
    elif i == '客户名称：':
        kehu = j
    else:
        continue
print(danhao,kehu)